In [4]:
!pip install transformers datasets torch scikit-learn pandas faiss-cpu sentence-transformers

import os, zipfile
import pandas as pd
import torch
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset

In [5]:
# Build FAISS index from retrieval headlines CSV
headlines_df = pd.read_csv("live_data_retrieval_headlines.csv")
live_headlines = headlines_df['Headline'].dropna().tolist()

# Embed & normalize
retrieval_model = SentenceTransformer("all-MiniLM-L6-v2")
embs = retrieval_model.encode(live_headlines, convert_to_numpy=True, show_progress_bar=True)
faiss.normalize_L2(embs)

# Create IndexFlatIP and add vectors
d = embs.shape[1]
faiss_index = faiss.IndexFlatIP(d)
faiss_index.add(embs)

print(f"Built FAISS index with {faiss_index.ntotal} vectors")

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

Built FAISS index with 2247 vectors


In [6]:
faiss.write_index(faiss_index, "faiss_live_headlines.index")
print("FAISS index saved as 'faiss_live_headlines.index'")

FAISS index saved as 'faiss_live_headlines.index'


In [7]:
# Load & preprocess live test dataset
test_df = pd.read_csv("politifact-sample.csv")
test_df = test_df.dropna(subset=['title','label'])
test_df['label'] = test_df['label'].astype(int)
test_df.reset_index(drop=True, inplace=True)
print(f"Loaded test set: {len(test_df)} samples")

Loaded test set: 2777 samples


In [8]:
# Retrieval + Input‐prep functions & Dataset class
def search_similar_articles(query_headline, model, faiss_index, headlines, k=3):
    q = query_headline.strip().lower()
    q_emb = model.encode([q], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    distances, indices = faiss_index.search(q_emb, k)
    results = []
    for rank, idx in enumerate(indices[0]):
        if idx < 0: continue
        results.append(f"{headlines[idx]} (dist: {distances[0][rank]:.4f})")
    return results

def prepare_input(article, facts, tokenizer, max_length=512):
    art_tokens = tokenizer.encode(article, add_special_tokens=False)
    input_ids = [tokenizer.cls_token_id] + art_tokens + [tokenizer.sep_token_id]
    tt = [0]*(len(art_tokens)+2)
    for fact in facts:
        f_tokens = tokenizer.encode(fact, add_special_tokens=False)
        input_ids += f_tokens + [tokenizer.sep_token_id]
        tt += [1]*(len(f_tokens)+1)
    am = [1]*len(input_ids)
    if len(input_ids)>max_length:
        input_ids, tt, am = input_ids[:max_length], tt[:max_length], am[:max_length]
    else:
        pad = max_length-len(input_ids)
        input_ids += [tokenizer.pad_token_id]*pad
        tt += [0]*pad
        am += [0]*pad
    return {
        'input_ids': torch.tensor(input_ids),
        'token_type_ids': torch.tensor(tt),
        'attention_mask': torch.tensor(am)
    }

class FakeNewsDataset(Dataset):
    def __init__(self, df, tokenizer, retrieval_model, faiss_index, headlines, max_length=512):
        self.df = df.reset_index(drop=True)
        self.tokenizer, self.rm, self.idx, self.hd = tokenizer, retrieval_model, faiss_index, headlines
        self.max_length = max_length
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        art, lbl = row['title'], row['label']
        facts = search_similar_articles(art, self.rm, self.idx, self.hd, k=3)
        enc = prepare_input(art, facts, self.tokenizer, self.max_length)
        enc['labels'] = torch.tensor(lbl, dtype=torch.long)
        return enc


In [9]:
# Unzip & load fine‐tuned RAG model
with zipfile.ZipFile("fine_tuned_bert_update_4_RAG.zip","r") as z:
    z.extractall("fine_tuned_bert_update_4_RAG")

model_dir = "fine_tuned_bert_update_4_RAG"
model = BertForSequenceClassification.from_pretrained(model_dir, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_dir)


In [10]:
# Prepare dataset & define metrics
test_dataset = FakeNewsDataset(test_df, tokenizer, retrieval_model, faiss_index, live_headlines)

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

args = TrainingArguments(
    output_dir="./eval_live",
    per_device_eval_batch_size=8,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [11]:
# Cell 8: Run evaluation
metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)

from sklearn.metrics import classification_report
preds_out = trainer.predict(test_dataset)
y_true, y_pred = preds_out.label_ids, np.argmax(preds_out.predictions, axis=-1)
print(classification_report(y_true, y_pred, target_names=["Fake","Real"]))

Evaluation metrics: {'eval_loss': 0.3867934048175812, 'eval_model_preparation_time': 0.0057, 'eval_accuracy': 0.9150162045372704, 'eval_runtime': 104.2114, 'eval_samples_per_second': 26.648, 'eval_steps_per_second': 3.339}
              precision    recall  f1-score   support

        Fake       0.94      0.97      0.95      2512
        Real       0.58      0.39      0.47       265

    accuracy                           0.92      2777
   macro avg       0.76      0.68      0.71      2777
weighted avg       0.90      0.92      0.91      2777



In [12]:
counts = test_df['label'].value_counts().sort_index()
print(f"Number of Fake examples (label 0): {counts.get(0, 0)}")
print(f"Number of Real examples (label 1): {counts.get(1, 0)}")

Number of Fake examples (label 0): 2512
Number of Real examples (label 1): 265
